ライブラリインポート

In [2]:
from collections import namedtuple
import csv
import math
import numpy as np

# 開口部の評価方法に関する仕様書

- LV2(#2) 計算方法名  
    - LV3(#3) 入力値  
    - LV3(#3) 出力値  
    - LV3(#3) 計算方法  
        - LV4(#4) 各計算方法  
    - LV3(#3) プログラムテスト  
        - LV4(#4) 入力  
        - LV4(#4) 出力  
        - LV4(#4) テストデータ  
        - LV4(#4) テスト結果

　本資料は開口部における熱貫流率および日射熱取得率を計算する方法についてまとめたものです。本資料は計算方法を示すとともに、プログラムのテストの入出力についてもまとめています。  
　本資料は以下のパートから構成されています。  
  
1. 気象データの読込  
2. 暖冷房期間の設定
3. 開口面日射量の計算  
　3.1. 太陽高度角・太陽方位角  
　3.2. 直散分離（Watanabe model）  
　3.3. 斜面日射量（Isotropic model）  
4. ひさしの日除け計算  
5. 窓の日射熱取得率の計算  
　5.1. グレージング複合体の日射熱取得率  
　　5.1.1. 面材の光学特性  
　　　5.1.1.1. ガラス  
　　　5.1.1.2. ロールスクリーン  
　　　5.1.1.3. ブラインド※  
　　　　5.1.1.3.1. 幾何学形状の作成※  
　　　　5.1.1.3.2. 鏡面反射成分の計算※  
　　　　5.1.1.3.3. 拡散反射成分の計算※  
　　　　5.1.1.3.4. 全体計算※  
　　5.1.2. 短波長放射（日射）の多重反射  
　　5.1.3. 熱平衡計算  
　　　5.1.3.1. 中空層の熱抵抗  
　　　5.1.3.2. 表面熱伝達抵抗  
　　　5.1.3.3. 全体計算  
　　5.1.4. 全体計算  
　5.2. フレームの日射熱取得率  
　5.3. 全体計算  
6. 窓の熱貫流率の計算※  

※は、将来的に記述するパートです。


## 1. 気象データの読込  

説明：気温と全天日射量を読み込む。

### 入力値  
w_location：地点名(このファイルと並列に置いた'…_tmp.csv'や'…_ig.csv'の'…'部分)   
'…_tmp.csv'：拡張アメダス気象データ1981-2000で出力した気温 (℃)  
'…_ig.csv'：拡張アメダス気象データ1981-2000で出力した全天日射量 [MJ/(m<sup>2</sup>･h)]  
w_lat：地点の緯度 (°)  
w_lon：地点の経度 (°)

### 出力値
'…_out.csv'：気象データのデータセット  
列1：各時刻のカウンタ（1～8760）  
列2：緯度 (°)  
列3：経度 (°)  
列4：各時刻の気温(℃)  
列5：各時刻の全天日射量 [MJ/(m<sup>2</sup>･h)]  

### 計算方法

"w_location"・"w_lat"・"w_lon"の組み合わせを1つの変数（タプル）で表し"weather_input"とする。  

In [52]:
weather_input = namedtuple('weather_input','w_location w_lat w_lon')
#w_location：地点名(このファイルと並列に置いた'…_tmp.csv'や'…_ig.csv'の'…'部分)   
#w_lat：地点の緯度 (°)  
#w_lon：地点の経度 (°)

In [53]:
#気象データセットの作成
def weather_data(L):
    tmp = np.genfromtxt(L.w_location + '_tmp.CSV', skip_header = 1, delimiter = ',', dtype = float)
    ig = np.genfromtxt(L.w_location + '_ig.CSV', skip_header = 1, delimiter = ',', dtype = float)
    w_dat = np.zeros((8760,5))
    for i in range(0, 365):
        for j in range(1, 25):
            w_dat[i * 24 + j - 1][0] = i * 24 + j
            w_dat[i * 24 + j - 1][1] = L.w_lat
            w_dat[i * 24 + j - 1][2] = L.w_lon
            w_dat[i * 24 + j - 1][3] = tmp[i, j]
            w_dat[i * 24 + j - 1][4] = ig[i, j]
    return w_dat

### プログラムテスト

In [57]:
def weather_TestImport(Ds):
    np.savetxt(Ds.w_location + '_out_1.csv', weather_data(Ds), fmt='%.10f', delimiter=',')

#### テストデータ

In [58]:
weather_inputDs = [weather_input('3639999', 35 + 41.2 / 60, 139 + 45.9 / 60)]

#### テスト結果
このファイルと並列の'…_out.csv'に出力される。

In [59]:
for d in weather_inputDs:
    weather_TestImport(d)

## 2. 暖冷房期間の設定  

（参考：平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）Ver.2.2 算定方法 1　概要と用語の定義）  

暖房期間は、日平均外気温をフーリエ変換した年周期成分が15℃以下となる期間とする。




冷房期間は、日最高外気温をフーリエ変換した年周期成分が23℃以上となる期間とする。


## 3. 開口面日射量の計算  

### 3.1. 太陽高度角・太陽方位角  

（参考：拡張アメダス気象データ1981-2000解説書 8.1　太陽位置の計算）  

赤緯と均時差の計算方法を以下に示す。  

$ \begin{align}n = Y - 1968\end{align}$ ･････････････････････････････････････････ (2.1-a)  

$ \begin{align}d_0 = 3.71 + 0.2596 ･ n - INT\biggl[\frac{n+3}{4}\biggr]\end{align}$ ･･････････････････ (2.1-b)  

$ \begin{align}M = \frac{360 ･ (D - d_0)}{D_y} \end{align}$ ････････････････････････････････････ (2.1-c)  

$ \begin{align}\epsilon = 12.3901 + 0.0172 ･ \biggl(n + \frac{M}{360}\biggr)\end{align}$ ･････････････････････ (2.1-d)  

$ \begin{align}v = M + 1.914 ･ \sin M + 0.02 ･ \sin(2M)\end{align}$ ･･････････････････ (2.1-e)  

$ \begin{align}E_t = (M - v) - \tan^{-1}\biggl[\frac{0.043 ･ \sin 2(v + \epsilon)}{1 - 0.043 ･ \cos 2(v + \epsilon)}\biggr]\end{align}$ ･････････ (2.1-f)  

$ \begin{align}\sin\delta = \cos(v + \epsilon) ･ \sin\delta_0\end{align}$ ･･･････････････････････････････ (2.1-g)  

ここで、  

$ n $ ：1968年との年差 (year)  
$ Y $ ：西暦年 (year)  
$ d_0 $ ：平均軌道上の近日点通過日（暦表時による1968年1月1日正午基準の日差）(day)  
$ INT[] $ ：小数点以下切り捨て  
$ M $ ：平均近点離角 (°)  
$ D $ ：1月1日を初日（$D=1$）とする計算対象日の年通算日 (day)  
$ D_y $ ：近点年（近日点基準の公転周期日数）（= 365.2596） (day)  
$ \epsilon $ ：近日点と冬至点の角度 (°)  
$ v $ ：真近点離角 (°)  
$ E_t $ ：均時差 (°)  
$ \delta $ ：赤緯 (°)  
$ \delta_0 $ ：北半球の冬至の日赤緯（= -23.4393） (°)  

である。  

また、太陽高度角と太陽方位角の計算方法を以下に示す。  

$ \begin{align}t = 15 ･ (T_m - 12) + (L - L_0) + E_t\end{align}$ ･･････････････････････ (2.1-h)  

$ \begin{align}\sin h = \sin\phi ･ \sin \delta + \cos \phi ･ \cos \delta ･ \cos t\end{align}$ ････････････････ (2.1-i)  

$ \begin{align}\sin A = \frac{\cos\delta ･ \sin t}{\cos h}\end{align}$ ････････････････････････････････････ (2.1-j)  

$ \begin{align}\cos A = \frac{\sin h ･ \sin\phi - \sin\delta}{\cos h ･ \cos\phi}\end{align}$ ････････････････････････････ (2.1-k)  

ここで、  

$ t $ ：時角 (°)  
$ T_m $ ：標準時 (h)  
$ L $ ：計算対象地点の経度 (°)  
$ L_0 $ ：標準時の地点の経度（=135.0（日本の場合）） (°)  
$ h $ ：太陽高度角 (°)  
$ \phi $ ：計算対象地点の緯度 (°)  
$ \delta $ ：赤緯 (°)  
$ A $ ：太陽方位角 (°)  

である。  

In [ ]:
#赤緯と均時差の計算プログラム(参考：拡張アメダス気象データ1981-2000解説書 8.1太陽位置の計算)
sol_n = sol_year - 1968.0
sol_d0 = 3.71 +　0.2596 * sol_n - int((sol_n + 3.0) / 4.0)
sol_Dy = 365.2596
sol_m = 360 * (sol_nday - sol_d0) / sol_Dy
sol_eps = 12.3901 + 0.0172 * (sol_n + sol_m / 360.0)
sol_v = sol_m + 1.914 * math.sin(math.radians(sol_m)) + 0.02 * math.sin(math.radians(2 * sol_m))
sol_veps = math.radians(sol_v + sol_eps)
sol_et = (sol_m - sol_v) - math.degrees(math.atan(0.043 * math.sin(2.0 * sol_veps) / (1.0 - 0.043 * math.cos(2.0 * sol_veps))))
sol_dlt0 = math.radians(-23.4393)
sol_sindlt = math.cos(veps) * math.sin(dlt0)
sol_cosdlt = (abs(1.0 - sindlt ** 2.0)) ** 0.5

#太陽位置の計算プログラム(参考：拡張アメダス気象データ1981-2000解説書 8.1太陽位置の計算)
sol_lons = 135.0
sol_t = 15.0 * (sol_tm - 12.0) + (sol_lon - sol_lons) + sol_et
sol_latrad = math.radians(sol_lat)
sol_trad = math.radians(sol_t)
sol_sinh = math.sin(sol_latrad) * sindlt + math.cos(sol_latrad) * cosdlt * math.cos(sol_trad)
sol_cosh = (1.0 - sol_sinh ** 2.0) ** 0.5
sol_sinA = sol_cosdlt * math.sin(sol_trad) / sol_cosh
sol_cosA = (sol_sinh * math.sin(sol_latrad) - sindlt) / (sol_cosh * math.cos(sol_latrad))


### 3.2. 直散分離（Watanabe model）

（参考：拡張アメダス気象データ1981-2000解説書 8.2.1　直散分離）  

直散分離（Watanabe model）の計算方法を以下に示す。  

$ \begin{align}I_d = I_0 ･ \sin h ･ \frac{Q}{1 + Q}\end{align}$ ･･････････････････････････････････ (2.2-a)  

$ \begin{align}Q = (0.8672 + 0.7505 ･ \sin h) ･ P ^ {\frac{0.421}{\sin h}} ･ (1 - P ^ {\frac{1}{\sin h}}) ^ {2.277}\end{align}$ ････ (2.2-b)  

$ \begin{align}P = \biggl(\frac{I_b}{I_0}\biggr) ^ {\sin h}\end{align}$ ････････････････････････････････････････ (2.2-c)  

$ \begin{align}I_b = \frac{I_G - I_d}{\sin h}\end{align}$ ･･････････････････････････････････････････ (2.2-d)  

ここで、  

$ I_G $ ：水平面全天日射量 [MJ/(m<sup>2</sup>･h)]  
$ I_b $ ：法線面直達日射量 [MJ/(m<sup>2</sup>･h)]  
$ I_d $ ：水平面天空日射量 [MJ/(m<sup>2</sup>･h)]  
$ I_0 $ ：法線面大気外日射量（=1367.0） [MJ/(m<sup>2</sup>･h)]  
$ P $ ：大気透過率  

である。  

In [ ]:
#直散分離（Watanabe model）
sol_i0 = 1367.0 #(W/m2)
sol_p = (sol_ib / sol_i0) ** sol_sinh
sol_q = (0.8672 + 0.7505 * sol_sinh) * sol_p ** (0.421 / sol_sinh) * (1 - sol_p ** (1 / sol_sinh)) ** 2.277
sol_id = sol_i0 * sol_sinh * sol_q / (1 + sol_q)
sol_ib = (sol_ig - sol_id) / sol_sinh
#収束計算

### 3.3. 斜面日射量（Isotropic model）

（参考：拡張アメダス気象データ1981-2000解説書 8.2.2　斜面日射量の計算）  

斜面日射量（Isotropic model）の計算方法を以下に示す。  

$ \begin{align}I_{T,b} = I_b ･ \cos i\end{align}$ ･････････････････････････････････････････ (2.3-a)  

$ \begin{align}I_{T,d} = I_d ･ \frac{1 + \cos\beta}{2}\end{align}$ ････････････････････････････････････ (2.3-b)  

$ \begin{align}I_{T,r} = I_G ･ \frac{1 - \cos\beta}{2} ･ \rho\end{align}$ ･････････････････････････････････ (2.3-c)  

ここで、  

$ I_{T,b} $ ：斜面直達日射量 [MJ/(m<sup>2</sup>･h)]  
$ I_{T,d} $ ：斜面天空日射量 [MJ/(m<sup>2</sup>･h)]  
$ I_{T,r} $ ：斜面地表面反射日射量 [MJ/(m<sup>2</sup>･h)]  
$ i $ ：法線面直達日射の入射角 (°)  
$ \beta $ ：水平面に対する斜面の傾斜角 (°)  
$ \rho $ ：アルベド  

である。  

In [2]:
#斜面日射量（Isotropic model）
sol_itb = sol_ib * sol_cosi
sol_itd = sol_id * (1 + math.cos(math.radians(sol_beta))) / 2
sol_itr = sol_ig * (1 - math.cos(math.radians(sol_beta))) / 2 * sol_rho

NameError: name 'sol_ib' is not defined

## 4. ひさしの日除け計算  

## 5. 窓の日射熱取得率の計算 

### 5.1. グレージング複合体の日射熱取得率  

### 5.1.1. 面材の光学特性

### 5.1.1.1. ガラス

説明：垂直入射時のガラスの日射透過率及び日射反射率から任意の入射角におけるガラスの日射透過率及び日射反射率を計算する。

### 入力値  
$ \tau_{0,f} $ ：ガラスの正面側からの入射光に対する垂直入射時の日射透過率  
$ \tau_{0,b} $ ：ガラスの背面側からの入射光に対する垂直入射時の日射透過率  
$ \rho_{0,f} $ ：ガラスの正面側からの入射光に対する垂直入射時の日射反射率  
$ \rho_{0,b} $ ：ガラスの背面側からの入射光に対する垂直入射時の日射反射率  
$ \phi$        ：日射の入射角（rad.）  
gtype          ：ガラスの種類flag（0：透明フロート板ガラス、1：Low-Eガラス）  
ctypef         ：正面側の膜の有無flag（0：膜なし、1：膜あり）  
ctypeb         ：背面側の膜の有無flag（0：膜なし、1：膜あり）  

### 出力値
$ \tau_{\phi,f} $ ：ガラスの正面側からの入射光に対する入射角$\phi$の日射透過率  
$ \tau_{\phi,b} $ ：ガラスの背面側からの入射光に対する入射角$\phi$の日射透過率  
$ \rho_{\phi,f} $ ：ガラスの正面側からの入射光に対する入射角$\phi$の日射反射率  
$ \rho_{\phi,b} $ ：ガラスの背面側からの入射光に対する入射角$\phi$の日射反射率

#### データ構造と値の範囲

$\tau_{0,f}$・$\tau_{0,b}$・$\rho_{0,f}$・$\rho_{0,b}$の組み合わせを1つの変数（タプル）で表し"glass_input"とする。  
全ての値は0.0以上1.0以下の値をとる。  
$\tau_{0,f}$ と $\rho_{0,f}$ の合計は1.0を超えない。  
$\tau_{0,b}$ と $\rho_{0,b}$ の合計は1.0を超えない。  
$\phi$は0.0以上 $\begin{align}\frac{\pi}{2}\end{align}$以下の値をとる。 

In [4]:
glass_input = namedtuple('glass_input','TRS0f TRS0b REF0f REF0b phi gtype ctypef ctypeb')
# TRS0f：正面側からの入射光に対する垂直入射時の日射透過率
# TRS0f：正面側からの入射光に対する垂直入射時の日射透過率
# TRS0b：背面側からの入射光に対する垂直入射時の日射透過率
# REF0f：正面側からの入射光に対する垂直入射時の日射反射率
# REF0b：背面側からの入射光に対する垂直入射時の日射反射率
# phi：日射の入射角（rad.）
# gtype：ガラスの種類flag（0：透明フロート板ガラス、1：Low-Eガラス）
# ctypef：正面側の膜の有無flag（0：膜なし、1：膜あり）
# ctypeb：背面側の膜の有無flag（0：膜なし、1：膜あり）

#### 入射角$\phi$の日射透過率及び日射反射率の計算

（参考：JIS A 2103：2014 附属書G 斜入射に対する板ガラスの光学特性の計算方法）  

入射角によって日射特性が変化するガラスの日射透過率及び日射反射率の計算方法を以下に示す。  

$ \begin{align}\tau_{\phi,f} = \tau_{0,f}･\sum_{i=0}^5 m_{i}･\cos^i\phi\end{align} $ ･････････････････････････････ (4.1.1.1-a)

$ \begin{align}\tau_{\phi,b} = \tau_{0,b}･\sum_{i=0}^5 m_{i}･\cos^i\phi\end{align} $ ･････････････････････････････ (4.1.1.1-b)

$ \begin{align}\rho_{\phi,f} =\rho_{0,f}+(1-\rho_{0,f})･\sum_{i=0}^5 m_{i}･\cos^i\phi\end{align} $ ･････････････････ (4.1.1.1-c)

$ \begin{align}\rho_{\phi,b} =\rho_{0,b}+(1-\rho_{0,b})･\sum_{i=0}^5 m_{i}･\cos^i\phi\end{align} $ ･････････････････ (4.1.1.1-d)  

ここで、  

$ m_{i} $ ：ガラス種類ごとの係数  

である。

In [1]:
# 入射角phiのglass_TRSf glass_TRSb glass_REFf glass_REFb を算出する。

def glass_TRSf(L):
    if L.TRS0f < 0.0:                   return ValueError
    elif L.TRS0f > 1.0:                 return ValueError 
    elif (L.TRS0f + L.REF0f) > 1.0:     return ValueError 
    elif L.phi <  0.0:                  return ValueError
    elif L.phi >  (math.pi / 2.0):      return ValueError
    else:
        glass_TRSnf = 0.0
        for i in range(0, 6):
            glass_TRSnf += glass_mTRS(L)[i] * math.cos(L.phi) ** i
        return L.TRS0f * glass_TRSnf

def glass_TRSb(L):
    if L.TRS0b < 0.0:                   return ValueError
    elif L.TRS0b > 1.0:                 return ValueError
    elif (L.TRS0b + L.REF0b) > 1.0:     return ValueError
    elif L.phi <  0.0:                  return ValueError
    elif L.phi >  (math.pi / 2.0):      return ValueError
    else:
        glass_TRSnb = 0.0
        for i in range(0, 6):
            glass_TRSnb += glass_mTRS(L)[i] * math.cos(L.phi) ** i
        return L.TRS0b * glass_TRSnb
    
def glass_REFf(L):
    if L.REF0f < 0.0:                   return ValueError
    elif L.REF0f > 1.0:                 return ValueError
    elif (L.TRS0f + L.REF0f) > 1.0:     return ValueError
    elif L.phi <  0.0:                  return ValueError
    elif L.phi >  (math.pi / 2.0):      return ValueError
    else:
        glass_REFnf = 0.0
        for i in range(0, 6):
            if L.ctypef == 0:
                glass_REFnf += glass_mREFg(L)[i] * math.cos(L.phi) ** i
            elif L.ctypef == 1:
                glass_REFnf += glass_mREFc(L)[i] * math.cos(L.phi) ** i

        return L.REF0f + (1 - L.REF0f) * glass_REFnf
    
def glass_REFb(L):
    if L.REF0b < 0.0:                   return ValueError
    elif L.REF0b > 1.0:                 return ValueError
    elif (L.TRS0b + L.REF0b) > 1.0:     return ValueError
    elif L.phi <  0.0:                  return ValueError
    elif L.phi >  (math.pi / 2.0):      return ValueError
    else:
        glass_REFnb = 0.0
        for i in range(0, 6):
            if L.ctypeb == 0:
                glass_REFnb += glass_mREFg(L)[i] * math.cos(L.phi) ** i
            elif L.ctypeb == 1:
                glass_REFnb += glass_mREFc(L)[i] * math.cos(L.phi) ** i

        return L.REF0b + (1 - L.REF0b) * glass_REFnb


$ m_{i} $ の係数表を以下に示す（参考：JIS A 2103：2014 表G.1）。なお、本仕様書では透明フロート板ガラス及びLow-Eガラスのみについてプログラムを記述する。  

| 板ガラス品種分類              | 規準化透過率又は規準化反射率 | $m_0$ | $m_1$ | $m_2$ | $m_3$ | $m_4$ | $m_5$ |
| :---                      | :---                         | ---: | ---: | ---: | ---: | ---: | ---: |
| 透明フロート板ガラス            | 規準化透過率                 | 0.000 | 2.552 | 1.364 | -11.388 | 13.617 | -5.146 |
|                               | 規準化反射率(ガラス面)       | 1.000 | -5.189 | 12.392 | -16.593 | 11.851 | -3.461 |
| 熱線吸収板ガラス              | 規準化透過率                 | 0.000 | 1.760 | 3.770 | -14.901 | 16.422 | -6.052 |
|                               | 規準化反射率(ガラス面)       | 1.000 | -5.189 | 12.392 | -16.593 | 11.851 | -3.461 |
| 熱線反射ガラス                | 規準化透過率                 | 0.000 | 3.297 | -1.122 | -8.408 | 12.206 | -4.972 |
|                               | 規準化反射率(ガラス面)       | 1.000 | -5.842 | 15.264 | -21.642 | 15.948 | -4.727 |
|                               | 規準化反射率(膜面)           | 1.000 | -5.842 | 15.264 | -21.642 | 15.948 | -4.727 |
| 熱線吸収熱線反射ガラス        | 規準化透過率                 | 0.000 | 2.558 | 0.989 | -11.214 | 14.275 | -5.608 |
|                               | 規準化反射率(ガラス面)       | 1.000 | -5.842 | 15.264 | -21.642 | 15.948 | -4.727 |
|                               | 規準化反射率(膜面)           | 1.000 | -5.842 | 15.264 | -21.642 | 15.948 | -4.727 |
| 高性能熱線反射ガラスSシリーズ | 規準化透過率                 | 0.000 | 5.342 | -10.170 | 6.670 | 0.886 | -1.728 |
|                               | 規準化反射率(ガラス面)       | 1.000 | -6.459 | 18.722 | -28.982 | 22.804 | -7.085 |
|                               | 規準化反射率(膜面)           | 1.000 | -9.187 | 33.681 | -59.593 | 50.413 | -16.314 |
| 高性能熱線反射ガラスTシリーズ | 規準化透過率                 | 0.000 | 2.552 | 1.364 | -11.388 | 13.617 | -5.146 |
|                               | 規準化反射率(ガラス面)       | 1.000 | -5.782 | 14.816 | -20.567 | 14.862 | -4.331 |
|                               | 規準化反射率(膜面)           | 1.000 | -8.900 | 31.953 | -55.934 | 47.050 | -15.176 |
| Low-Eガラス                    | 規準化透過率                 | 0.000 | 2.273 | 1.631 | -10.358 | 11.769 | -4.316 |
|                               | 規準化反射率(ガラス面)       | 1.000 | -5.084 | 12.646 | -18.213 | 13.967 | -4.316 |
|                               | 規準化反射率(膜面)           | 1.000 | -4.387 | 9.175 | -11.152 | 7.416 | -2.052 |


In [6]:
#係数mを選択する。ここでは、透明フロート板ガラス及びLow-Eガラスのみを記述する。

def glass_mTRS(L):
    if L.gtype == 0:    return [0.000, 2.552, 1.364, -11.388, 13.617, -5.146]
    elif L.gtype == 1:  return [0.000, 2.273, 1.631, -10.358, 11.769, -4.316]
    else:               return ValueError

def glass_mREFg(L):
    if L.gtype == 0:    return [1.000, -5.189, 12.392, -16.593, 11.851, -3.461]
    elif L.gtype == 1:  return [1.000, -5.084, 12.646, -18.213, 13.967, -4.316]
    else:               return ValueError

def glass_mREFc(L):
    if L.gtype == 0:    return ValueError
    elif L.gtype == 1:  return [1.000, -4.387, 9.175, -11.152, 7.416, -2.052]
    else:               return ValueError


### プログラムテスト

#### 入力

入力[1]： ガラスの正面側からの入射光に対する垂直入射時の日射透過率  
入力[2]： ガラスの背面側からの入射光に対する垂直入射時の日射透過率  
入力[3]： ガラスの正面側からの入射光に対する垂直入射時の日射反射率  
入力[4]： ガラスの背面側からの入射光に対する垂直入射時の日射反射率  
入力[5]： 日射の入射角（rad.）
入力[6]： ガラスの種類flag（0：透明フロート板ガラス、1：Low-Eガラス）  
入力[7]： 正面側の膜の有無flag（0：膜なし、1：膜あり）  
入力[8]： 背面側の膜の有無flag（0：膜なし、1：膜あり）  

#### 出力

出力[1]：ガラスの正面側からの入射光に対する入射角$\phi$の日射透過率  
出力[2]：ガラスの背面側からの入射光に対する入射角$\phi$の日射透過率  
出力[3]：ガラスの正面側からの入射光に対する入射角$\phi$の日射反射率  
出力[4]：ガラスの背面側からの入射光に対する入射角$\phi$の日射反射率

In [7]:
def glass_TestAngTrsRef(Ds):
    
    glass_RV = []
    
    # 正面側からの入射光に対する入射角φの日射透過率
    glass_RV.append(glass_TRSf(Ds))
    # 背面側からの入射光に対する入射角φの日射透過率
    glass_RV.append(glass_TRSb(Ds))
    # 正面側からの入射光に対する入射角φの日射反射率
    glass_RV.append(glass_REFf(Ds))
    # 背面側からの入射光に対する入射角φの日射反射率
    glass_RV.append(glass_REFb(Ds))
        
    # テスト結果の出力
    print (glass_RV)

#### テストデータ

In [8]:
glass_inputDs = [glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 0, 0, 0, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 1, 0, 0, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 2, 0, 0, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 3, 0, 0, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 4, 0, 0, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 5, 0, 0, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 6, 0, 0, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 7, 0, 0, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 8, 0, 0, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 9, 0, 0, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 0, 1, 1, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 1, 1, 1, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 2, 1, 1, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 3, 1, 1, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 4, 1, 1, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 5, 1, 1, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 6, 1, 1, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 7, 1, 1, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 8, 1, 1, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 9, 1, 1, 0),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 0, 1, 0, 1),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 1, 1, 0, 1),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 2, 1, 0, 1),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 3, 1, 0, 1),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 4, 1, 0, 1),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 5, 1, 0, 1),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 6, 1, 0, 1),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 7, 1, 0, 1),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 8, 1, 0, 1),
                 glass_input(0.815, 0.815, 0.072, 0.072, math.pi / 18 * 9, 1, 0, 1)]

#### テスト結果

In [9]:
for d in glass_inputDs:
    glass_TestAngTrsRef(d)

[0.8141850000000012, 0.8141850000000012, 0.07200000000000041, 0.07200000000000041]
[0.8155349768594915, 0.8155349768594915, 0.07301961913715516, 0.07301961913715516]
[0.8151494736460383, 0.8151494736460383, 0.07445869495324935, 0.07445869495324935]
[0.8061308214289546, 0.8061308214289546, 0.074712501879066, 0.074712501879066]
[0.7887349149655521, 0.7887349149655521, 0.07743033550193344, 0.07743033550193344]
[0.7656289581956525, 0.7656289581956525, 0.09163574879437847, 0.09163574879437847]
[0.7202562500000003, 0.7202562500000003, 0.12944899999999998, 0.12944899999999998]
[0.602302695567371, 0.602302695567371, 0.21764921207768106, 0.21764921207768106]
[0.3555188030622195, 0.3555188030622195, 0.4394415797851921, 0.4394415797851921]
[1.2735591923052985e-16, 1.2735591923052985e-16, 0.9999999999999997, 0.9999999999999997]
[0.8141849999999997, 0.8141849999999997, 0.07200000000000205, 0.07200000000000081]
[0.8143612568331651, 0.8143612568331651, 0.07319582612105147, 0.07374424606059289]
[0.811

### 5.1.1.2. ロールスクリーン

説明：垂直入射時のロールスクリーンの日射透過率及び日射反射率から任意の入射角におけるロールスクリーンの日射透過率及び日射反射率を計算する。

### 入力値  
$ \tau_{0,f} $ ：ロールスクリーンの正面側からの入射光に対する垂直入射時の日射透過率  
$ \tau_{0,b} $ ：ロールスクリーンの背面側からの入射光に対する垂直入射時の日射透過率  
$ \rho_{0,f} $ ：ロールスクリーンの正面側からの入射光に対する垂直入射時の日射反射率  
$ \rho_{0,b} $ ：ロールスクリーンの背面側からの入射光に対する垂直入射時の日射反射率  
$ \phi$        ：日射の入射角（rad.）

### 出力値
$ \tau_{\phi,f} $ ：ロールスクリーンの正面側からの入射光に対する入射角$\phi$の日射透過率  
$ \tau_{\phi,b} $ ：ロールスクリーンの背面側からの入射光に対する入射角$\phi$の日射透過率  
$ \rho_{\phi,f} $ ：ロールスクリーンの正面側からの入射光に対する入射角$\phi$の日射反射率  
$ \rho_{\phi,b} $ ：ロールスクリーンの背面側からの入射光に対する入射角$\phi$の日射反射率

### 計算方法

#### データ構造と値の範囲

$\tau_{0,f}$・$\tau_{0,b}$・$\rho_{0,f}$・$\rho_{0,b}$の組み合わせを1つの変数（タプル）で表し"role_input"とする。  
全ての値は0.0以上1.0以下の値をとる。  
$\tau_{0,f}$ と $\rho_{0,f}$ の合計は1.0を超えない。  
$\tau_{0,b}$ と $\rho_{0,b}$ の合計は1.0を超えない。  
$\phi$は0.0以上$ \begin{align}\frac {\pi}{2}\end{align}$以下の値をとる。  

In [10]:
role_input = namedtuple('role_input','TRS0f TRS0b REF0f REF0b phi')
# TRS0f：正面側からの入射光に対する垂直入射時の日射透過率
# TRS0b：背面側からの入射光に対する垂直入射時の日射透過率
# REF0f：正面側からの入射光に対する垂直入射時の日射反射率
# REF0b：背面側からの入射光に対する垂直入射時の日射反射率
# phi：日射の入射角（rad.）

#### 入射角φの日射透過率及び日射反射率の計算

入射角によって日射特性が変化しないロールスクリーンの日射透過率及び日射反射率の計算方法を以下に示す。  

$ \begin{align}\tau_{\phi,f} = \tau_{0,f}\end{align}$ ････････････････････････････････････････････ (4.1.1.2-a)

$ \begin{align}\tau_{\phi,b} = \tau_{0,b}\end{align}$ ････････････････････････････････････････････ (4.1.1.2-b)

$ \begin{align}\rho_{\phi,f} =\rho_{0,f}\end{align}$ ････････････････････････････････････････････ (4.1.1.2-c)

$ \begin{align}\rho_{\phi,b} =\rho_{0,b}\end{align}$ ････････････････････････････････････････････ (4.1.1.2-d)  

In [11]:
# 入射角phiのrole_TRSf role_TRSb role_REFf role_REFb を算出する。

def role_TRSf(L):
    if L.TRS0f < 0.0:                   return ValueError
    elif L.TRS0f > 1.0:                 return ValueError 
    elif (L.TRS0f + L.REF0f) > 1.0:     return ValueError 
    elif L.phi <  0.0:                  return ValueError
    elif L.phi >  (math.pi / 2.0):      return ValueError
    else:                               return L.TRS0f

def role_TRSb(L):
    if L.TRS0b < 0.0:                   return ValueError
    elif L.TRS0b > 1.0:                 return ValueError
    elif (L.TRS0b + L.REF0b) > 1.0:     return ValueError
    elif L.phi <  0.0:                  return ValueError
    elif L.phi >  (math.pi / 2.0):      return ValueError
    else:                               return L.TRS0b
    
def role_REFf(L):
    if L.REF0f < 0.0:                   return ValueError
    elif L.REF0f > 1.0:                 return ValueError
    elif (L.TRS0f + L.REF0f) > 1.0:     return ValueError
    elif L.phi <  0.0:                  return ValueError
    elif L.phi >  (math.pi / 2.0):      return ValueError
    else:                               return L.REF0f
    
def role_REFb(L):
    if L.REF0b < 0.0:                   return ValueError
    elif L.REF0b > 1.0:                 return ValueError
    elif (L.TRS0b + L.REF0b) > 1.0:     return ValueError
    elif L.phi <  0.0:                  return ValueError
    elif L.phi >  (math.pi / 2.0):      return ValueError
    else:                               return L.REF0b


### プログラムテスト

#### 入力

入力[1]： ロールスクリーンの正面側からの入射光に対する垂直入射時の日射透過率  
入力[2]： ロールスクリーンの背面側からの入射光に対する垂直入射時の日射透過率  
入力[3]： ロールスクリーンの正面側からの入射光に対する垂直入射時の日射反射率  
入力[4]： ロールスクリーンの背面側からの入射光に対する垂直入射時の日射反射率  
入力[5]： 日射の入射角（rad.）

#### 出力

出力[1]：ロールスクリーンの正面側からの入射光に対する入射角$\phi$の日射透過率  
出力[2]：ロールスクリーンの背面側からの入射光に対する入射角$\phi$の日射透過率  
出力[3]：ロールスクリーンの正面側からの入射光に対する入射角$\phi$の日射反射率  
出力[4]：ロールスクリーンの背面側からの入射光に対する入射角$\phi$の日射反射率

In [12]:
def role_TestAngTrsRef(Ds):
    
    role_RV = []
    
    # ロールスクリーンの正面側からの入射光に対する入射角φの日射透過率
    role_RV.append(role_TRSf(Ds))
    # ロールスクリーンの背面側からの入射光に対する入射角φの日射透過率
    role_RV.append(role_TRSb(Ds))
    # ロールスクリーンの正面側からの入射光に対する入射角φの日射反射率
    role_RV.append(role_REFf(Ds))
    # ロールスクリーンの背面側からの入射光に対する入射角φの日射反射率
    role_RV.append(role_REFb(Ds))
        
    # テスト結果の出力
    print (role_RV)

#### テストデータ

In [13]:
role_inputDs = [ role_input(0.9, 0.5, 0.05, 0.3, 0.0),
                 role_input(0.9, 0.5, 0.05, 0.3, 0.5),
                 role_input(0.9, 0.5, 0.05, 0.3, 1.0),
                 role_input(0.9, 0.5, 0.05, 0.3, 1.5),
                 role_input(-0.5, 0.5, 0.05, 0.3, 0.5),
                 role_input(0.9, -0.5, 0.05, 0.3, 0.5), 
                 role_input(0.9, 0.5, -0.5, 0.3, 0.5),
                 role_input(0.9, 0.5, 0.05, -0.5, 0.5),
                 role_input(0.9, 0.5, 0.05, 0.3, -0.5) ]

#### テスト結果

In [14]:
for d in role_inputDs:
    role_TestAngTrsRef(d)

[0.9, 0.5, 0.05, 0.3]
[0.9, 0.5, 0.05, 0.3]
[0.9, 0.5, 0.05, 0.3]
[0.9, 0.5, 0.05, 0.3]
[<class 'ValueError'>, 0.5, 0.05, 0.3]
[0.9, <class 'ValueError'>, 0.05, 0.3]
[0.9, 0.5, <class 'ValueError'>, 0.3]
[0.9, 0.5, 0.05, <class 'ValueError'>]
[<class 'ValueError'>, <class 'ValueError'>, <class 'ValueError'>, <class 'ValueError'>]


### 5.1.1.3. ブラインド

### 5.1.1.3.1. 幾何学形状の作成

### 5.1.1.3.2. 鏡面反射成分の計算

### 5.1.1.3.3. 拡散反射成分の計算

### 5.1.1.3.4. 全体計算

### 5.1.2. 短波長放射（日射）の多重反射 

説明：グレージング・日除け・付属部材等の各層において、短波長日射の正面（室外）側からの透過率、背面（室内）側からの透過率、正面側からの反射率、背面側からの反射率が既知の場合に、短波長の多重反射を解いて、グレージング複合体全体の透過率および、各層で吸収される短波長の吸収率を計算する。添え字$i$は層の番号（0～$n$-1）であり、$n$は層の数を表す。

### 入力値  
$ \tau_{f,j} $ ：層jの正面側からの入射光に対する日射透過率  
$ \tau_{b,j} $ ：層jの背面側からの入射光に対する日射透過率  
$ \rho_{f,j} $ ：層jの正面側からの入射光に対する日射反射率  
$ \rho_{b,j} $ ：層jの背面側からの入射光に対する日射反射率

### 出力値
$ \alpha_j $ ：層jの正面側からの入射光に対する日射吸収率

### 計算方法

#### 層$j$のデータ構造と値の範囲

層1つにつき、$\tau_f$・$\tau_b$・$\rho_f$・$\rho_b$の組み合わせを1つの変数（タプル）で表し"layer"とする。  
全ての値は0.0以上1.0以下の値をとる。
$\tau_f$ と $\rho_f$ の合計は1.0を超えない。  
$\tau_b$ と $\rho_b$ の合計は1.0を超えない。

In [9]:
layer = namedtuple('glazing','TRSf TRSb REFf REFb')
# TRSf：正面側からの入射光に対する日射透過率
# TRSf：正面側からの入射光に対する日射透過率
# TRSb：背面側からの入射光に対する日射透過率
# REFf：正面側からの入射光に対する日射反射率
# REFb：背面側からの入射光に対する日射反射率

In [10]:
# データが適切な範囲に収まっているかをチェック
def CheckLayerSpec(L):
    if(L.TRSf < 0): raise ValueError
    if(L.TRSb < 0): raise ValueError
    if(L.REFf < 0): raise ValueError
    if(L.REFb < 0): raise ValueError
    if(L.TRSf + L.REFf > 1.0): raise ValueError
    if(L.TRSb + L.REFb > 1.0): raise ValueError

#### 任意の層から任意の層までの多重反射計算

（参考：JIS A 2103：2014 5.4.2 日射透過率及び各面材の日射吸収率）

隣り合う層$i$から層$j$までの値の計算方法を以下に示す。  

$ \begin{align}\tau_{f,i,j} = \frac {\tau_{f,i,j-1}･\tau_{f,j}}{1-\rho_{b,i,j-1}･\rho_{f,j}}\end{align}$ ････････････････････････････････ (4.1.2-a)

$ \begin{align}\tau_{b,i,j} = \frac {\tau_{b,j}･\tau_{b,i,j-1}}{1-\rho_{b,i,j-1}･\rho_{f,j}}\end{align}$ ････････････････････････････････ (4.1.2-b)

$ \begin{align}\rho_{f,i,j} = \rho_{f,i,j-1} + \frac {\tau_{f,i,j-1}･\rho_{f,j}･\tau_{b,i,j-1}}{1-\rho_{b,i,j-1}･\rho_{f,j}}\end{align}$ ･････････････････････ (4.1.2-c)

$ \begin{align}\rho_{b,i,j} = \rho_{b,j} + \frac {\tau_{b,j}･\rho_{b,i,j-1}･\tau_{f,j}}{1-\rho_{b,i,j-1}･\rho_{f,j}}\end{align}$ ･･････････････････････････ (4.1.2-d)

ここで、

$ \tau_{f,j} $　：　層$j$の正面側からの入射光に対する日射透過率  
$ \tau_{b,j} $　：　層$j$の背面側からの入射光に対する日射透過率  
$ \rho_{f,j} $　：　層$j$の正面側からの入射光に対する日射反射率  
$ \rho_{b,j} $　：　層$j$の背面側からの入射光に対する日射反射率  
$ \tau_{f,i,j} $　：　層$i$－$j$間の正面側からの入射光に対する日射透過率  
$ \tau_{b,i,j} $　：　層$i$－$j$間の背面側からの入射光に対する日射透過率  
$ \rho_{f,i,j} $　：　層$i$－$j$間の正面側からの入射光に対する日射反射率  
$ \rho_{b,i,j} $　：　層$i$－$j$間の背面側からの入射光に対する日射反射率  

である。  
任意の層$i$－$j$間の値を求める場合は、層$i+1$, $i+2$,..., $j-2$, $j-1$, $j$というように、式(4.1.2-a)～式(4.1.2-d)を逐次代入して求める。  
ここで、$ i=j $ のときは、

$ \begin{align}\tau_{f,i,j} = \tau_{f,j}\end{align} $ ････････････････････････････････････････････ (4.1.2-e)

$ \begin{align}\tau_{b,i,j} = \tau_{b,j}\end{align} $ ････････････････････････････････････････････ (4.1.2-f)

$ \begin{align}\rho_{f,i,j} = \rho_{f,j}\end{align} $ ････････････････････････････････････････････ (4.1.2-g)

$ \begin{align}\rho_{b,i,j} = \rho_{b,j}\end{align} $ ････････････････････････････････････････････ (4.1.2-h)

である。

In [11]:
# 層riから層rjの複合した、TRSf TRSb REFf REFb を算出する。
def SolarRadAbsComp(L,ri,rj):
    
    # riとrjの数値の範囲に関するエラー処理
    if ri <  0:      raise ValueError
    if rj <  0:      raise ValueError
    if ri >  rj:     raise ValueError
    if rj >= len(L): raise ValueError

    if ri == rj:
        return L[ri]
    else:
        SRAC = SolarRadAbsComp(L, ri, rj-1)
        trsf =              ( SRAC.TRSf  * L[rj].TRSf              ) / ( 1 - SRAC.REFb * L[rj].REFf )
        trsb =              ( L[rj].TRSb * SRAC.TRSb               ) / ( 1 - SRAC.REFb * L[rj].REFf )
        reff = SRAC.REFf  + ( SRAC.TRSf  * L[rj].REFf * SRAC.TRSb  ) / ( 1 - SRAC.REFb * L[rj].REFf )
        refb = L[rj].REFb + ( L[rj].TRSb * SRAC.REFb  * L[rj].TRSf ) / ( 1 - SRAC.REFb * L[rj].REFf )
        return layer( trsf, trsb, reff, refb )


#### 各層の日射吸収率

（参考：JIS A 2103：2014 5.4.2 日射透過率及び各面材の日射吸収率）

各層の値の計算方法を以下に示す。  
（JISでは添え字は1始まりであるが、本仕様書では0始まりとしている。） 

$ j = 0 $ かつ $ n = 0 $ の場合

$ \begin{align} \alpha_j = \alpha_{f,j} \end{align} $ ･･････････････････････････････････････････････ (4.1.2-i)

$ j = 0 $ かつ $ n ≠ 0 $ の場合

$ \begin{align} \alpha_j = \alpha_{f,j} + \frac {\tau_{f,0,j}･\rho_{f,j+1,n}}{1-\rho_{b,0,j}･\rho_{f,j+1,n}}･\alpha_{b,j} \end{align} $ ･･････････････････････ (4.1.2-j)

$ j = 1 $～$ n - 2 $ の場合  

$ \begin{align} \alpha_j = \frac {\tau_{f,0,j-1}}{1-\rho_{b,0,j-1}･\rho_{f,j,n}}･\alpha_{f,j} + \frac {\tau_{f,0,j}･\rho_{f,j+1,n}}{1-\rho_{b,0,j}･\rho_{f,j+1,n}}･\alpha_{b,j} \end{align} $ ････ (4.1.2-k)

$ j = n - 1 $ の場合

$ \begin{align} \alpha_j = \frac {\tau_{f,0,j-1}}{1-\rho_{b,0,j-1} \rho_{f,j,n}}･\alpha_{f,j} \end{align} $ ･････････････････････････････ (4.1.2-l)

ここで、

$ \alpha_j $ ：層jの面材の日射吸収率  
$ j $ ：層の番号  
$ n $ ：層の数  

である。  
また、

$ \begin{align} \alpha_{f,j} = 1 - \tau_{f,j} - \rho_{f,j} \end{align} $ ･･･････････････････････････････････ (4.1.2-m)

$ \begin{align} \alpha_{b,j} = 1 - \tau_{b,j} - \rho_{b,j} \end{align} $ ･･･････････････････････････････････ (4.1.2-n)

である。

In [17]:
# 層iの日射吸収率を算出する。
def SolarRadAbsLayer(L,rj):
    
    # 層の正面側からの入射光に対する日射吸収率
    def Absf(lyr):
        return 1.0 - lyr.TRSf - lyr.REFf
    
    # 層の背面側からの入射光に対する日射吸収率
    def Absb(lyr):
        return 1.0 - lyr.TRSb - lyr.REFb
    
    # 層の最背面の番号
    n = len(L) - 1
    
    # 関数リネーム
    SRAC = SolarRadAbsComp
    
    # 層jの正面側からの入射光
    def SolarRad_f(L,rj):
        if rj == 0:
            return 1.0
        else:
            return SRAC(L,0,rj-1).TRSf / ( 1 - SRAC(L,0,rj-1).REFb * SRAC(L,rj,n).REFf )
            
    # 層jの背面側からの入射光
    def SolarRad_b(L,rj,n):
        if rj == n:
            return 0.0
        else:
            return SRAC(L,0,rj).TRSf * SRAC(L,rj+1,n).REFf / ( 1 - SRAC(L,0,rj).REFb * SRAC(L,rj+1,n).REFf )
        
    # rjの数値の範囲に関するエラー処理
    if rj < 0: raise ValueError
    if rj > n: raise ValueError

    return SolarRad_f(L,rj) * Absf(L[rj]) + SolarRad_b(L,rj,n) * Absb(L[rj])


#### グレージング複合体全体の透過率

グレージング複合体全体の透過率は、定義から、

$ \begin{align} \tau_{glz} = \tau_{f,0,n-1} \end{align} $ ･････････････････････････････････････････ (4.1.2-o)

となる。  
ここで、

$ \tau_{glz} $ ：グレージング複合体全体の透過率

である。

In [18]:
# グレージング複合体全体の透過率
def SolarRadTransAll(L):
    return SolarRadAbsComp(L,0,len(L)-1).TRSf

### プログラムテスト

#### 入力

入力[0] : layer の数  
入力[4j+1]： 層jの正面側からの入射光に対する日射透過率  
入力[4j+2]： 層jの背面側からの入射光に対する日射透過率  
入力[4j+3]： 層jの正面側からの入射光に対する日射反射率  
入力[4j+4]： 層jの背面側からの入射光に対する日射反射率  

#### 出力

出力1   ： グレージング複合体全体の正面側の反射率  
出力2   ： グレージング複合体全体の透過率  
出力3[j]： 層jの日射吸収率  
出力4   ： グレージング複合体全体の正面側の反射率・グレージング複合体全体の透過率・層jの日射吸収率の合計  
※　出力1は全体計算には使わない  
※　出力4は必ず1.0になる

In [19]:
def TestSolarRadAbs_Trans(Ds):
    
    RV = []
    
    # グレージング複合体全体の正面側の反射率
    RV.append(SolarRadAbsComp(Ds,0,len(Ds)-1).REFf)

    # グレージング複合体全体の透過率
    RV.append(SolarRadTransAll(Ds))

    # 層jの日射吸収率
    for j in range(len(Ds)):
        RV.append(SolarRadAbsLayer(Ds,j))

    # グレージング複合体全体の正面側の反射率・グレージング複合体全体の透過率・層jの日射吸収率の合計
    RV.append(sum(RV))
    
    # テスト結果の出力
    print (RV)

#### テストデータ

| 製品名称                                                    | 記号         | TRSf  | TRSb  | REFf  | REFb  |
| --------                                                    | ----         | ----  | ----  | ----  | ----  |
| 透明フロートガラス　FL6                                     | FL6          | 0.815 | 0.815 | 0.072 | 0.072 |
| 網入り磨き板ガラス　PW6.8                                   | PW6.8        | 0.732 | 0.732 | 0.075 | 0.075 |
| Low-E　日射遮蔽型　LQ6                                      | LQ6          | 0.396 | 0.396 | 0.316 | 0.442 |
| Low-E日射取得型　LS6                                        | LS6          | 0.571 | 0.571 | 0.207 | 0.246 |
| 熱線反射ガラス　SKFC6                                       | SKFC6        | 0.648 | 0.648 | 0.230 | 0.262 |
| 熱線吸収熱線反射ガラス　SBRKEC6                             | SBRKFC6      | 0.394 | 0.394 | 0.103 | 0.244 |
| 高性能熱線反射ガラスSシリーズ　SGY32-6                      | SGY32-6      | 0.302 | 0.302 | 0.108 | 0.248 |
| 高性能熱線反射ガラスTシリーズ　TS30-6                       | TS30-6       | 0.241 | 0.241 | 0.154 | 0.312 |
| スクリーンホワイト                                          | RSWH         | 0.300 | 0.300 | 0.630 | 0.630 |
| スクリーングレイ                                            | RSGR         | 0.230 | 0.230 | 0.520 | 0.520 |
| スクリーンセピア                                            | RSSP         | 0.180 | 0.180 | 0.370 | 0.370 |
| ブラインドホワイト25mm幅プロファイル0度スラット0度(水平）   | BRWH25_0-0   | 0.915 | 0.915 | 0.045 | 0.045 |
| ブラインドホワイト25mm幅プロファイル0度スラット45度         | BRWH25_0-45  | 0.308 | 0.306 | 0.391 | 0.360 |
| ブラインドホワイト25mm幅プロファイル0度スラット75度(全閉）  | BRWH25_0-75  | 0.048 | 0.038 | 0.646 | 0.635 |
| ブラインドグレイ35mm幅プロファイル30度スラット0度(水平）    | BRGR35_30-0  | 0.448 | 0.448 | 0.147 | 0.147 |
| ブラインドグレイ35mm幅プロファイル30度スラット45度          | BRGR35_30-45 | 0.057 | 0.673 | 0.390 | 0.099 |
| ブラインドグレイ35mm幅プロファイル30度スラット75度(全閉）   | BRGR35_30-75 | 0.016 | 0.091 | 0.503 | 0.379 |
| ブラインドブラック50mm幅プロファイル60度スラット0度(水平）  | BRBR50_60-0  | 0.001 | 0.001 | 0.014 | 0.014 |
| ブラインドブラック50mm幅プロファイル60度スラット45度        | BRBR50_60-45 | 0.000 | 0.415 | 0.048 | 0.001 |
| ブラインドブラック50mm幅プロファイル60度スラット75度(全閉） | BRBR50_60-75 | 0.000 | 0.393 | 0.050 | 0.004 |

| No. | Comp.                       | Layer | TRSf  | TRSb  | REFf  | REFb  | TRSf  | TRSb  | REFf  | REFb  | TRSf  | TRSb  | REFf  | REFb  |
| --- | ---                         | ----- | ----  | ----  |       |       |       |       |       |       |       |       |       |       |
| 1   | FL6                         | 1     | 0.815 | 0.815 | 0.072 | 0.072 |       |       |       |       |       |       |       |       |
| 2   | PW6.8                       | 1     | 0.732 | 0.732 | 0.075 | 0.075 |       |       |       |       |       |       |       |       |
| 3   | SKFC6                       | 1     | 0.648 | 0.648 | 0.230 | 0.262 |       |       |       |       |       |       |       |       |
| 4   | SBRKFC6                     | 1     | 0.394 | 0.394 | 0.103 | 0.244 |       |       |       |       |       |       |       |       |
| 5   | SGY32-6                     | 1     | 0.302 | 0.302 | 0.108 | 0.248 |       |       |       |       |       |       |       |       |
| 6   | TS30-6                      | 1     | 0.241 | 0.241 | 0.154 | 0.312 |       |       |       |       |       |       |       |       |
| 7   | FL6＋A+FL6                  | 2     | 0.815 | 0.815 | 0.072 | 0.072 | 0.815 | 0.815 | 0.072 | 0.072 |       |       |       |       |
| 8   | FL6+A+PW6.8                 | 2     | 0.815 | 0.815 | 0.072 | 0.072 | 0.732 | 0.732 | 0.075 | 0.075 |       |       |       |       |
| 9   | LQ6+A＋FL6                  | 2     | 0.396 | 0.396 | 0.316 | 0.442 | 0.815 | 0.815 | 0.072 | 0.072 |       |       |       |       |
| 10  | FL6+A+LS6                   | 2     | 0.815 | 0.815 | 0.072 | 0.072 | 0.571 | 0.571 | 0.246 | 0.207 |       |       |       |       |
| 11  | SKFC6＋A+FL6                | 2     | 0.648 | 0.648 | 0.230 | 0.262 | 0.815 | 0.815 | 0.072 | 0.072 |       |       |       |       |
| 12  | SBRKFC6+A+FL6               | 2     | 0.394 | 0.394 | 0.103 | 0.244 | 0.815 | 0.815 | 0.072 | 0.072 |       |       |       |       |
| 13  | SGY32-6+A+FL6               | 2     | 0.302 | 0.302 | 0.108 | 0.248 | 0.815 | 0.815 | 0.072 | 0.072 |       |       |       |       |
| 14  | TS30-6+A+FL6                | 2     | 0.241 | 0.241 | 0.154 | 0.312 | 0.815 | 0.815 | 0.072 | 0.072 |       |       |       |       |
| 15  | FL6+A+RSWH                  | 2     | 0.815 | 0.815 | 0.072 | 0.072 | 0.300 | 0.300 | 0.630 | 0.630 |       |       |       |       |
| 16  | PW6.8+A+RSSP                | 2     | 0.732 | 0.732 | 0.075 | 0.075 | 0.180 | 0.180 | 0.370 | 0.370 |       |       |       |       |
| 17  | SKFC6＋A+BRWH25_0-0         | 2     | 0.648 | 0.648 | 0.230 | 0.262 | 0.915 | 0.915 | 0.045 | 0.045 |       |       |       |       |
| 18  | SBRKFC6＋A+BRWH25_0-75      | 2     | 0.394 | 0.394 | 0.103 | 0.244 | 0.048 | 0.038 | 0.646 | 0.635 |       |       |       |       |
| 19  | SGY32-6+A+BRGR35_30-45      | 2     | 0.302 | 0.302 | 0.108 | 0.248 | 0.057 | 0.673 | 0.390 | 0.099 |       |       |       |       |
| 20  | TS30-6+A+BRBR50_60-45       | 2     | 0.241 | 0.241 | 0.154 | 0.312 | 0.000 | 0.415 | 0.048 | 0.001 |       |       |       |       |
| 21  | FL6+A+FL6+RSGR              | 3     | 0.815 | 0.815 | 0.072 | 0.072 | 0.815 | 0.815 | 0.072 | 0.072 | 0.230 | 0.230 | 0.520 | 0.520 |
| 22  | LQ6+A+FL6+BRWH25_0-45       | 3     | 0.396 | 0.396 | 0.316 | 0.442 | 0.815 | 0.815 | 0.072 | 0.072 | 0.308 | 0.306 | 0.391 | 0.360 |
| 23  | FL6+A+LS6+BRGR35_30-0       | 3     | 0.815 | 0.815 | 0.072 | 0.072 | 0.571 | 0.571 | 0.246 | 0.207 | 0.448 | 0.448 | 0.147 | 0.147 |
| 24  | LS6+A+FL6+BRGR35_30-75      | 3     | 0.571 | 0.571 | 0.207 | 0.246 | 0.815 | 0.815 | 0.072 | 0.072 | 0.016 | 0.091 | 0.503 | 0.379 |
| 25  | SBRKFC6＋A+FL6+BRBR50_60-0  | 3     | 0.394 | 0.394 | 0.103 | 0.244 | 0.815 | 0.815 | 0.072 | 0.072 | 0.001 | 0.001 | 0.014 | 0.014 |
| 26  | SGY32-6＋A+FL6+BRBR50_60-75 | 3     | 0.302 | 0.302 | 0.108 | 0.248 | 0.815 | 0.815 | 0.072 | 0.072 | 0.000 | 0.393 | 0.050 | 0.004 |

In [187]:
inputDs = [ [layer(0.815,0.815,0.072,0.072)],
            [layer(0.732,0.732,0.075,0.075)],
            [layer(0.648,0.648,0.23 ,0.262)],
            [layer(0.394,0.394,0.103,0.244)],
            [layer(0.302,0.302,0.108,0.248)],
            [layer(0.241,0.241,0.154,0.312)],
            [layer(0.815,0.815,0.072,0.072),layer(0.815,0.815,0.072,0.072)],
            [layer(0.815,0.815,0.072,0.072),layer(0.732,0.732,0.075,0.075)],
            [layer(0.396,0.396,0.316,0.442),layer(0.815,0.815,0.072,0.072)],
            [layer(0.815,0.815,0.072,0.072),layer(0.571,0.571,0.246,0.207)],
            [layer(0.648,0.648,0.23 ,0.262),layer(0.815,0.815,0.072,0.072)],
            [layer(0.394,0.394,0.103,0.244),layer(0.815,0.815,0.072,0.072)],
            [layer(0.302,0.302,0.108,0.248),layer(0.815,0.815,0.072,0.072)],
            [layer(0.241,0.241,0.154,0.312),layer(0.815,0.815,0.072,0.072)],
            [layer(0.815,0.815,0.072,0.072),layer(0.3  ,0.3  ,0.63 ,0.63 )],
            [layer(0.732,0.732,0.075,0.075),layer(0.18 ,0.18 ,0.37 ,0.37 )],
            [layer(0.648,0.648,0.23 ,0.262),layer(0.915,0.915,0.045,0.045)],
            [layer(0.394,0.394,0.103,0.244),layer(0.048,0.038,0.646,0.635)],
            [layer(0.302,0.302,0.108,0.248),layer(0.057,0.673,0.39 ,0.099)],
            [layer(0.241,0.241,0.154,0.312),layer(0    ,0.415,0.048,0.001)],
            [layer(0.815,0.815,0.072,0.072),layer(0.815,0.815,0.072,0.072),layer(0.23 ,0.23 ,0.52 ,0.52 )],
            [layer(0.396,0.396,0.316,0.442),layer(0.815,0.815,0.072,0.072),layer(0.308,0.306,0.391,0.36 )],
            [layer(0.815,0.815,0.072,0.072),layer(0.571,0.571,0.246,0.207),layer(0.448,0.448,0.147,0.147)],
            [layer(0.571,0.571,0.207,0.246),layer(0.815,0.815,0.072,0.072),layer(0.016,0.091,0.503,0.379)],
            [layer(0.394,0.394,0.103,0.244),layer(0.815,0.815,0.072,0.072),layer(0.001,0.001,0.014,0.014)],
            [layer(0.302,0.302,0.108,0.248),layer(0.815,0.815,0.072,0.072),layer(0    ,0.393,0.05 ,0.004)] ]


#### テスト結果

In [188]:
for d in inputDs:
    TestSolarRadAbs_Trans(d)

[0.072, 0.815, 0.11300000000000006, 1.0]
[0.075, 0.732, 0.193, 1.0]
[0.23, 0.648, 0.12199999999999997, 1.0]
[0.103, 0.394, 0.503, 1.0]
[0.108, 0.302, 0.59, 1.0]
[0.154, 0.241, 0.605, 1.0]
[0.12007341257076684, 0.6676862857050951, 0.11966539339938245, 0.09257490832475557, 1.0]
[0.12208734667202895, 0.5998190227227026, 0.11994462598029365, 0.15814900462497486, 1.0]
[0.32766187965824395, 0.3333484820941647, 0.29277076895109977, 0.04621886929649157, 0.9999999999999999]
[0.23834566440799437, 0.4737561692701121, 0.1360638773964459, 0.15183428892544756, 0.9999999999999999]
[0.2608143702809804, 0.5382740007501509, 0.12627977365013612, 0.0746318553187326, 1.0]
[0.11437686068857691, 0.32685213836682847, 0.5134528516986417, 0.04531814924595293, 1.0]
[0.11468607454711326, 0.2506047992962335, 0.5999626938615925, 0.034746432295060616, 0.9999999999999999]
[0.1582779314521409, 0.20092866145083146, 0.6129345865523111, 0.027858820544716525, 1.0]
[0.5103450829632112, 0.25611748931534395, 0.17377668021453

### 5.1.3. 熱平衡計算  

### 5.1.3.1. 中空層の熱抵抗

説明：面材間の表面温度から中空層の熱抵抗を計算する。

### 入力値  
$ \theta_1 $ ：面1の表面温度 (℃)  
$ \theta_2 $ ：面2の表面温度 (℃)  
$ \epsilon_1 $ ：面1の修正放射率  
$ \epsilon_2 $ ：面2の修正放射率  
$ s $ ：中空層の厚さ (m)  
gas_type_1：気体1の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）  
gas_type_2：気体2の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）  
gas_ratio_1：気体1の容積割合（0.0～1.0）  
gas_ratio_2：気体2の容積割合（1.0 - gas_ratio_1）  
gas_dir：中空層の熱流方向flag（0：中空層が垂直で熱流方向が水平、1：中空層が水平で熱流方向が上向き、2：中空層が45度で熱流方向が上向き）    

### 出力値
$ R_{s} $ ：中空層の熱抵抗 (m<sup>2</sup>･K/W)  

#### データ構造と値の範囲

$ \theta_1 $・$ \theta_2 $・$ \epsilon_1 $・$ \epsilon_2 $・$ s $・gas_type_1・gas_type_2・gas_ratio_1・gas_dirの組み合わせを1つの変数（タプル）で表し"gas_input"とする。  

In [9]:
gas_input = namedtuple('gas_input','Tmc_1 Tmc_2 eps_1 eps_2 gas_s gas_type_1 gas_type_2 gas_ratio_1 gas_dir')

# Tmc_1：面1の表面温度（℃）
# Tmc_2：面2の表面温度（℃）
# eps_1：面1の修正放射率
# eps_2：面2の修正放射率
# gas_s：中空層の厚さ（m）
# gas_type_1：気体1の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）
# gas_type_2：気体2の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）
# gas_ratio_1：気体1の容積割合（0.0～1.0）
# gas_dir：中空層の熱流方向flag（0：中空層が垂直で熱流方向が水平、1：中空層が水平で熱流方向が上向き、2：中空層が45度で熱流方向が上向き）

#### 中空層の熱抵抗

（参考：JIS R 3107：1998 4.1 基礎式）  

中空層の熱抵抗の計算方法を以下に示す。 

$ \begin{align}R_{s} = \frac{1}{h_r+h_g}\end{align}$ ････････････････････････････････････････ (4.1.3.1-a)  

ここで、  

$ R_{s} $ ：中空層の熱抵抗 (m<sup>2</sup>･K/W)  
$ h_{r} $ ：中空層における放射熱コンダクタンス [W/(m<sup>2</sup>･K)]  
$ h_{g} $ ：中空層における気体の伝導と対流による熱コンダクタンス（気体熱コンダクタンス）[W/(m<sup>2</sup>･K)]  

である。 

#### 中空層の放射熱コンダクタンス

（参考：JIS R 3107：1998 4.2 放射熱コンダクタンス）  

中空層における放射熱コンダクタンスの計算方法を以下に示す。

$ \begin{align}h_r = 4･\sigma･(\frac{1}{\epsilon_1}+\frac{1}{\epsilon_2}-1)^{-1}･T_m^3\end{align}$ ･･･････････････････････ (4.1.3.1-b)  


ここで、  

$ \sigma $ ：ステファン・ボルツマン定数（＝5.67×10<sup>-7</sup>）[W/(m<sup>2</sup>･K<sup>4</sup>)]  
$ \epsilon_1 $，$ \epsilon_2 $ ：中空層に接する二つのガラス面の修正放射率  
$ T_m $ ：中空層に接する二つのガラス面の絶対温度の平均値 (K)  

である。  

#### 中空層の気体熱コンダクタンス

（参考：JIS R 3107：1998 4.3 気体熱コンダクタンス）  

中空層における気体熱コンダクタンスの計算方法を以下に示す。

$ \begin{align}h_g = Nu･\frac{\lambda}{s}\end{align}$ ･･････････････････････････････････････････ (4.1.3.1-c)

$ \begin{align}Nu = A･(Gr･Pr)^n\end{align}$ ････････････････････････････････････ (4.1.3.1-d)

$ \begin{align}Gr = \frac{9.81･s^3･\Delta T･\rho^2}{T'_m･\mu^2}\end{align}$ ････････････････････････････････ (4.1.3.1-e)

$ \begin{align}Pr = \frac{\mu･c}{\lambda}\end{align}$ ････････････････････････････････････････････ (4.1.3.1-f)  

ここで、  

$ Nu $ ：ヌセルト数  
$ \lambda $ ：気体の熱伝導率 [W/(m･K)]  
$ s $ ：中空層の厚さ (m)  
$ A $，$ n $ ：中空層の傾斜角と熱流方向によって決まる数値  
$ Gr $ ：グラスホフ数  
$ Pr $ ：プラントル数  
$ \Delta T $ ：中空層に接する二つのガラス面の温度差 (K)  
$ \rho $ ：気体の密度 (kg/m<sup>3</sup>)  
$ T'_m $ ：中空層の気体の平均絶対温度 (K)  
$ \mu $ ：気体の粘度 [kg/(m･s)]  
$ c $ ：気体の比熱 [J/(kg･K)]  

である。

中空層の傾斜角と熱流方向によって決まる数値$ A $，$ n $は、以下の値とする（参考：JIS R3107：1998 5.3.1）。

| 条件 | $A$ | $n$ |
| :--- | ---: | ---: |
| 中空層が垂直で熱流方向が水平の場合 | 0.035 | 0.38 |
| 中空層が水平で熱流方向が上向きの場合 | 0.16 | 0.28 |
| 中空層が45°で熱流方向が上向きの場合 | 0.10 | 0.31 |

また、混合気体の場合には、気体の物性値は、式(4.1.3-g)によって計算する（参考：JIS R3107：1998 5.3.2）。

$ \begin{align}F = \sum_{i=1}^N F_{i}･r_{i}\end{align} $ ･････････････････････････････････････････ (4.1.3.1-g)  

ここで、  

$ F $ ：混合気体の密度、粘度、熱伝導率又は比熱  
$ F_{i} $ ：各気体の密度、粘度、熱伝導率又は比熱  
$ r_{i} $ ：各気体の容積割合  
$ N $ ：気体の数  

である。なお、本仕様書では、$N$は1または2とする。

In [10]:
def gas_R(L):
    gas_ratio_2 = 1.0 - L.gas_ratio_1
    gas_dT = abs(L.Tmc_1 - L.Tmc_2)
    gas_Tmc = (L.Tmc_1 + L.Tmc_2) / 2.0
    gas_Tm = gas_Tmc + 273.15
    ave_Tm = gas_Tm
    
    if L.gas_dir == 0:
        gas_A = 0.035
        gas_n = 0.38
    elif L.gas_dir == 1:
        gas_A = 0.16
        gas_n = 0.28
    elif L.gas_dir == 2:
        gas_A = 0.10
        gas_n = 0.31
    
    sgm = 5.67 * 10.0 ** (-8.0)
    
    for j in range(1, 5):
        gas_mix = gas_prop(L.gas_type_1, gas_Tmc, j) * L.gas_ratio_1 + gas_prop(L.gas_type_2, gas_Tmc, j) * gas_ratio_2
        if j == 1: gas_rho = gas_mix
        elif j == 2: gas_mu = gas_mix * 10.0 ** (-5.0)
        elif j == 3: gas_lmd = gas_mix * 10.0 ** (-2.0)
        elif j == 4: gas_c = gas_mix * 10.0 ** 3.0
    gas_Gr = 9.81 * L.gas_s ** 3.0 * gas_dT * gas_rho ** 2.0 / (gas_Tm * gas_mu ** 2.0)
    gas_Pr = gas_mu * gas_c / gas_lmd
    gas_Nu = gas_A * (gas_Gr * gas_Pr)** gas_n
    if gas_Nu <= 1.0: gas_Nu = 1.0
    gas_hg = gas_Nu * gas_lmd / L.gas_s
    gas_hr = 4.0 * sgm * (1.0 / L.eps_1 + 1.0 / L.eps_2 -1.0) ** (-1.0) * ave_Tm ** 3.0
    return 1.0 / (gas_hr + gas_hg)


気体の物性値を以下に示す（参考：JIS R 3107：1998 付表2）。  

| 気体 | 温度 $\theta$ (℃) | 密度 $\rho$ (kg/m<sup>3</sup>) | 粘度 $\mu$ [kg/(m･s)] | 熱伝導率 $\lambda$ [W/(m･K)] | 比熱 $c$ [J/(kg･K)] |
| :--- | ---: | ---: | ---: | ---: | ---: |
| 空気 | -10.0 | 1.326 | 1.661e-5 | 2.336e-2 | 1.008e+3 |
|      | 0.0   | 1.277 | 1.711e-5 | 2.416e-2 | 1.008e+3 |
|      | 10.0  | 1.232 | 1.761e-5 | 2.496e-2 | 1.008e+3 |
|      | 20.0  | 1.189 | 1.811e-5 | 2.576e-2 | 1.008e+3 |
| アルゴン | -10.0 | 1.829 | 2.038e-5 | 1.584e-2 | 0.519e+3 |
|          | 0.0   | 1.762 | 2.101e-5 | 1.634e-2 | 0.519e+3 |
|          | 10.0  | 1.699 | 2.164e-5 | 1.684e-2 | 0.519e+3 |
|          | 20.0  | 1.640 | 2.228e-5 | 1.734e-2 | 0.519e+3 |
| SF<sub>6</sub> | -10.0 | 6.844 | 1.383e-5 | 1.119e-2 | 0.614e+3 |
|                | 0.0   | 6.602 | 1.421e-5 | 1.197e-2 | 0.614e+3 |
|                | 10.0  | 6.360 | 1.459e-5 | 1.275e-2 | 0.614e+3 |
|                | 20.0  | 6.118 | 1.497e-5 | 1.354e-2 | 0.614e+3 |
| クリプトン | -10.0 | 3.832 | 2.260e-5 | 0.842e-2 | 0.245e+3 |
|            | 0.0   | 3.690 | 2.330e-5 | 0.870e-2 | 0.245e+3 |
|            | 10.0  | 3.560 | 2.400e-5 | 0.900e-2 | 0.245e+3 |
|            | 20.0  | 3.430 | 2.470e-5 | 0.926e-2 | 0.245e+3 |

In [11]:
def gas_prop(gas_type, gas_Tmc, j):
    if gas_type == 0: #空気
        gp = np.array([[-10.0, 1.326, 1.661, 2.336, 1.008], [0.0, 1.277, 1.711, 2.416, 1.008],
                       [10.0, 1.232, 1.761, 2.496, 1.008], [20.0, 1.189, 1.811, 2.576, 1.008]])
    
    elif gas_type == 1: #アルゴン
        gp = np.array([[-10.0, 1.829, 2.038, 1.584, 0.519], [0.0, 1.762, 2.101, 1.634, 0.519],
                       [10.0, 1.699, 2.164, 1.684, 0.519], [20.0, 1.640, 2.228, 1.734, 0.519]])

    elif gas_type == 2: #SF6
        gp = np.array([[-10.0, 6.844, 1.383, 1.119, 0.614], [0.0, 6.602, 1.421, 1.197, 0.614],
                       [10.0, 6.360, 1.459, 1.275, 0.614], [20.0, 6.118, 1.497, 1.354, 0.614]])        
 
    elif gas_type == 3: #クリプトン
        gp = np.array([[-10.0, 3.832, 2.260, 0.842, 0.245], [0.0, 3.690, 2.330, 0.870, 0.245],
                       [10.0, 3.560, 2.400, 0.900, 0.245], [20.0, 3.430, 2.470, 0.926, 0.245]])         
    
    for i in range(1, 4):
        if i == 1 and gas_Tmc < gp[i-1, 0]: break
        elif gp[i-1, 0] <= gas_Tmc and gas_Tmc < gp[i, 0]: break
        elif i == 3 and gp[i, 0] <= gas_Tmc: break
       
    gp_cal = gp[i-1, j] + (gp[i, j] - gp[i-1, j]) * (gas_Tmc - gp[i-1, 0]) / (gp[i, 0] - gp[i-1, 0])
    
    return gp_cal


### プログラムテスト

#### 入力

入力[1]： 面1の表面温度 (℃)  
入力[2]： 面2の表面温度 (℃)  
入力[3]： 面1の修正放射率  
入力[4]： 面2の修正放射率  
入力[5]： 中空層の厚さ (m)  
入力[6]： 気体1の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）  
入力[7]： 気体2の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）  
入力[8]： 気体1の容積割合（0.0～1.0）  
入力[9]： 中空層の熱流方向flag（0：中空層が垂直で熱流方向が水平、1：中空層が水平で熱流方向が上向き、2：中空層が45度で熱流方向が上向き）  

#### 出力

出力[1]：中空層の熱抵抗 (m<sup>2</sup>･K/W)  

In [15]:
def gas_TestRsst(Ds):
    
    gas_RV = []
    
    # 中空層の熱抵抗
    gas_RV.append(gas_R(Ds))
        
    # テスト結果の出力
    print (gas_RV)

#### テストデータ

In [16]:
gas_inputDs = [gas_input(0.0, 20.0, 0.837, 0.837, 0.012, 0, 0, 1, 0),
               gas_input(30.0, 25.0, 0.837, 0.837, 0.012, 0, 0, 1, 0),
               gas_input(0.0, 20.0, 0.837, 0.837, 0.012, 1, 0, 0.8, 0) ]

#### テスト結果

In [17]:
for d in gas_inputDs:
    gas_TestRsst(d)

[0.17284803028788262]
[0.15077336074792078]
[0.19069064931470364]


### 5.1.3.2. 表面熱伝達抵抗

説明：表面温度と放射温度から表面熱伝達抵抗を計算する。

### 入力値  
$ \theta_{sur} $ ：表面温度 (℃)  
$ \theta_{r} $ ：放射温度 (℃)  
$ \epsilon_{sur} $ ：表面放射率   
cnv_type：対流熱伝達のflag（0：夏期室内、1：夏期屋外、2：冬期室内、3：冬期屋外）  

### 出力値
$ R_{s} $ ：中空層の熱抵抗 (m<sup>2</sup>･K/W)  

#### データ構造と値の範囲

$ \theta_{sur} $・$ \theta_{r} $・$ \epsilon_{sur} $・cond_typeの組み合わせを1つの変数（タプル）で表し"surface_input"とする。  

In [26]:
surface_input = namedtuple('surface_input','Tmc_sur Tmc_r eps_sur cnv_type')

# Tmc_sur：表面温度（℃）
# Tmc_r：放射温度（℃）
# eps_sur：表面放射率
# cnv_type：対流熱伝達のflag（0：夏期室内、1：夏期屋外、2：冬期室内、3：冬期屋外）

（参考：JIS A 2103：2014 6. 境界条件）  

表面熱伝達抵抗の計算方法を以下に示す。 

$ \begin{align}R_{sur} = \frac{1}{h_{r,sur}+h_{c,sur}}\end{align}$ ･･････････････････････････････････ (4.1.3.2-a)  

$ \begin{align}h_{r,sur} = \frac{\epsilon_{sur}･\sigma･(T^4_{sur}-T^4_{r})}{T_{sur}-T_{r}}\end{align}$ ････････････････････････････ (4.1.3.2-b)  

ここで、  

$ R_{sur} $ ：表面熱伝達抵抗 (m<sup>2</sup>･K/W)  
$ h_{r,sur} $ ：表面放射熱伝達率 [W/(m<sup>2</sup>･K)]  
$ h_{c,sur} $ ：表面対流熱伝達率[W/(m<sup>2</sup>･K)]  
$ \epsilon_{sur} $ ：表面放射率  
$ \sigma $ ：ステファン・ボルツマン定数（＝5.67×10<sup>-7</sup>）[W/(m<sup>2</sup>･K<sup>4</sup>)]  
$ T_{sur} $ ：表面温度（$T_{sur}=\theta_{sur}+273.15$） (K)   
$ T_{r} $ ：放射温度（$T_{r}=\theta_{r}+273.15$）(K)   

である。なお、本仕様書では、放射温度と空気温度は等しいものとして扱う。

また、表面対流熱伝達率は、季節や室内外条件に応じた以下の値とする（参考：JIS A 2103：2014 表4）。

| 季節 | 室内外 | 表面対流熱伝達率 $h_{c,sur}$ [W/(m<sup>2</sup>･K)] |
| :---: | :---: | ---: |
| 夏期 | 室内 | 2.5 |
|  | 屋外 | 8.0 |
| 冬期 | 室内 | 3.6 |
|  | 屋外 | 20.0 |

In [31]:
def surface_R(L):

    Tm_sur = L.Tmc_sur + 273.15
    Tm_r = L.Tmc_r + 273.15
    sgm = 5.67 * 10.0 ** (-8.0)
    sur_hr = L.eps_sur * sgm * (Tm_sur ** 4 - Tm_r ** 4) / (Tm_sur - Tm_r)
    
    if L.cnv_type == 0:
        sur_hc = 2.5
    elif L.cnv_type == 1:
        sur_hc = 8.0
    elif L.cnv_type == 2:
        sur_hc = 3.6
    elif L.cnv_type == 3:
        sur_hc = 20.0
        
    return 1.0 / (sur_hr + sur_hc)


### プログラムテスト

#### 入力

入力[1]： 面1の表面温度 (℃)  
入力[2]： 面2の表面温度 (℃)  
入力[3]： 面1の修正放射率  
入力[4]： 面2の修正放射率  
入力[5]： 中空層の厚さ (m)  
入力[6]： 気体1の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）  
入力[7]： 気体2の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）  
入力[8]： 気体1の容積割合（0.0～1.0）  
入力[9]： 中空層の熱流方向flag（0：中空層が垂直で熱流方向が水平、1：中空層が水平で熱流方向が上向き、2：中空層が45度で熱流方向が上向き）  

#### 出力

出力[1]：中空層の熱抵抗 (m<sup>2</sup>･K/W)  

In [32]:
def surface_TestRsst(Ds):
    
    sur_RV = []
    
    # 中空層の熱抵抗
    sur_RV.append(surface_R(Ds))
        
    # テスト結果の出力
    print (sur_RV)

#### テストデータ

In [33]:
sur_inputDs = [surface_input(16.0, 20.0, 0.837, 2),
               surface_input(4.0, 0.0, 0.837, 3) ]

#### テスト結果

In [34]:
for d in sur_inputDs:
    surface_TestRsst(d)

[0.12069537873775033]
[0.04174567607094187]


### 5.1.3.3. 全体計算

説明：グレージング複合体の熱平衡モデルによって面材間の表面温度と熱抵抗を計算する。

（参考：JIS A 2103：2014 5.4.4 面材間の中空層の熱抵抗）  

各面材間における日射吸収と熱伝達との熱平衡の計算方法を以下に示す。

$ \begin{align}\frac{I･\alpha_j}{2} = \frac{\theta_{f,j} - \theta_{b,j-1}}{R_{j-1,j}} + \frac{\theta_{f,j} - \theta_{b,j}}{R_{j}}\end{align}$ ･･････････････････････ (4.1.3-h)  

$ \begin{align}\frac{I･\alpha_j}{2} = \frac{\theta_{b,j} - \theta_{f,j}}{R_{j}} + \frac{\theta_{b,j} - \theta_{f,j+1}}{R_{j,j+1}} \end{align}$ ･･････････････････････ (4.1.3-i)  

ここで、  

$ J_j $ ：層$j$の面材の吸収日射熱量 (W/m<sup>2</sup>)  
$ \alpha_j $ ：層$j$の面材の日射吸収率  
$ \theta_{f,j} $ ：層$j$の面材の正面側表面温度 (℃)  
$ \theta_{b,j} $ ：層$j$の面材の背面側表面温度 (℃)  
$ R_j $ ：層$j$の面材の熱抵抗 (m<sup>2</sup>･K/W)  
$ R_{j-1,j} $ ：層$j-1$と層$j$の面材間の中空層の熱抵抗 (m<sup>2</sup>･K/W) 

である。  
また、  
$ \begin{align}J_j = I_{dir,j}･\alpha_{dir,j} + I_{sky,j}･\alpha_{sky,j} + I_{ref,j}･\alpha_{ref,j} \end{align}$ ･･････････････････････ (4.1.3-j)

### 5.1.4. 全体計算

### 5.2. フレームの日射熱取得率

### 5.3. 全体計算

## 6. 窓の熱貫流率の計算